# Generate transitions

In [ ]:
# To run in in Google Colab, uncomment the following:

# !pip install expertsystem graphviz

A {doc}`Partial Wave Analysis <pwa:index>` starts by defining an amplitude model that describes the reaction process that is to be analyzed. Such a model is generally very complex and requires a fair amount of effort by the analyst (you). This gives a lot of room for mistakes.

The [‘expert system’](https://en.wikipedia.org/wiki/Expert_system) is responsible to give you advice on the form of an amplitude model, based on the problem set you define (initial state, final state, allowed interactions, intermediate states, etc.). Internally, the system propagates the quantum numbers through the reaction graph while satisfying the specified conservation rules. How to control this procedure is explained in more detail below.

Afterwards, the amplitude model of the expert system can be exported into [Tensorwaves](https://pwa.readthedocs.io/projects/tensorwaves). The model can for instance be used to generate a data set (toy Monte Carlo) for this reaction and to optimize its parameters to resemble an actual data set as good as possible. For more info on that see {doc}`amplitude`.

```{note}
Simple channels can be treated with the {func}`.generate_transitions` façade function. This notebook shows how to treat more complicated cases with the {class}`.StateTransitionManager`.
```

## 1. Define the problem set

We first define the boundary conditions of our physics problem, such as initial state, final state, formalism type, etc. and pass all of that information to the {class}`.StateTransitionManager`. This is the main user interface class of the {mod}`expertsystem`.

```{toggle}
By default, the {class}`.StateTransitionManager` loads all particles from the PDG. The {mod}`expertsystem` would take a long time to check the quantum numbers of all these particles, so in this notebook, we use a smaller subset of relatively common particles.
```

````{margin}
```{hint}
How does the {class}`.StateTransitionManager` know what a `"J/psi(1S)"` is? Upon construction, the {class}`.StateTransitionManager` loads default definitions from the [PDG](https://pdg.lbl.gov). See {doc}`/usage/particle` for how to add custom particle definitions.
```
````

In [ ]:
from expertsystem.reaction import InteractionTypes, StateTransitionManager

stm = StateTransitionManager(
    initial_state=["J/psi(1S)"],
    final_state=["gamma", "pi0", "pi0"],
    formalism_type="helicity",
)

```{eval-rst}
.. admonition:: `.StateTransitionManager`
  :class: dropdown

  The `.StateTransitionManager` (STM) is the main user interface class of the {mod}`expertsystem`. The boundary conditions of your physics problem, such as the initial state, final state, formalism type, etc., are defined through this interface.

  * `.create_problem_sets` of the STM creates all problem sets ― using the boundary conditions of the `.StateTransitionManager` instance. In total 3 steps are performed. The creation of reaction topologies. The creation of `.InitialFacts`, based on a topology and the initial and final state information. And finally the solving settings such as the conservation laws and quantum number domains to use at which point of the topology.

  * By default, all three interaction types (`~.InteractionTypes.EM`, `~.InteractionTypes.STRONG`, and `~.InteractionTypes.WEAK`) are used in the preparation stage. However, it is also possible to choose the allowed interaction types globally via `.set_allowed_interaction_types`.

  After the preparation step, you can modify the problem sets returned by `.create_problem_sets` to your liking. Since the output of this function contains quite a lot of information, the `expertsystem` aids in the configuration (especially the STM).

  * A subset of particles that are allowed as intermediate states can also be specified: either through the `STM's constructor <.StateTransitionManager>` or by setting the instance :code:`allowed_intermediate_particles`.
```

## 2. Prepare Problem Sets

Create all {class}`.ProblemSet`'s using the boundary conditions of the {class}`.StateTransitionManager` instance. By default it uses the **isobar model** (tree of two-body decays) to build {class}`.Topology`'s. Various {class}`.InitialFacts` are created for each topology based on the initial and final state. Lastly some reasonable default settings for the solving process are chosen. Remember that each interaction node defines its own set of conservation laws.

The {class}`.StateTransitionManager` (STM) defines three interaction types:

| Interaction          | Strength  |
|:-------------------- |:--------- |
| strong               | $60$      |
| electromagnetic (EM) | $1$       |
| weak                 | $10^{-4}$ |

By default, all three are used in the preparation stage. The {meth}`~.StateTransitionManager.create_problem_sets` method of the STM generates graphs with all possible combinations of interaction nodes. An overall interaction strength is assigned to each graph and they are grouped according to this strength.

In [ ]:
problem_sets = stm.create_problem_sets()

## 3. Find solutions

If you are happy with the default settings generated by the {class}`.StateTransitionManager`, just start with solving directly!

```{toggle}
This step takes about 23 sec on an Intel(R) Core(TM) i7-6820HQ CPU of 2.70GHz running, multi-threaded.
```

In [ ]:
result = stm.find_solutions(problem_sets)

The {meth}`~.StateTransitionManager.find_solutions` method returns a {class}`.Result` object from which you can extract the {attr}`~.Result.transitions`. Now, you can use {meth}`~.Result.get_intermediate_particles` to print the names of the intermediate states that the {class}`.StateTransitionManager` found:

In [ ]:
print("found", len(result.transitions), "solutions!")
result.get_intermediate_particles().names

````{admonition} About the number of solutions
---
class: dropdown
----

The "number of {attr}`~.Result.transitions`" is the total number of allowed {obj}`.StateTransitionGraph` instances that the {class}`.StateTransitionManager` has found. This also includes all allowed **spin projection combinations**. In this channel, we for example consider a $J/\psi$ with spin projection $\pm1$ that decays into a $\gamma$ with spin projection $\pm1$, which already gives us four possibilities.

On the other hand, the intermediate state names that was extracted with {meth}`.Result.get_intermediate_particles`, is just a {obj}`set` of the state names on the intermediate edges of the list of {attr}`~.Result.transitions`, regardless of spin projection.
````

Now we have a lot of solutions that are actually heavily suppressed (they involve two weak decays).

In general, you can modify the {class}`.ProblemSet`s returned by {meth}`~.StateTransitionManager.create_problem_sets` directly, but the STM also comes with functionality to globally choose the allowed interaction types. So, go ahead and **disable** the {attr}`~.InteractionTypes.EM` and {attr}`.InteractionTypes.WEAK` interactions:

In [ ]:
stm.set_allowed_interaction_types([InteractionTypes.STRONG])
problem_sets = stm.create_problem_sets()
result = stm.find_solutions(problem_sets)

print("found", len(result.transitions), "solutions!")
result.get_intermediate_particles().names

Now note that, since a $\gamma$ particle appears in one of the interaction nodes, the expert system knows that this node **must involve EM interactions**! Because the node can be an effective interaction, the weak interaction cannot be excluded, as it contains only a subset of conservation laws.

Since only the strong interaction was supposed to be used, this results in a warning and the STM automatically corrects the mistake.

Once the EM interaction is included, this warning disappears. Be aware, however, that the EM interaction is now available globally. Hence, there now might be solutions in which both nodes are electromagnetic.

In [ ]:
stm.set_allowed_interaction_types(
    [InteractionTypes.STRONG, InteractionTypes.EM]
)
problem_sets = stm.create_problem_sets()
result = stm.find_solutions(problem_sets)

print("found", len(result.transitions), "solutions!")
result.get_intermediate_particles().names

Great! Now we selected only the strongest contributions. Be aware, though, that there are more effects that can suppress certain decays, like small branching ratios. In this example, the initial state $J/\Psi$ can decay into $\pi^0 + \rho^0$ or $\pi^0 + \omega$.

| decay                   | branching ratio |
|:----------------------- |:--------------- |
| $\omega\to\gamma+\pi^0$ | 0.0828          |
| $\rho^0\to\gamma+\pi^0$ | 0.0006          |

Unfortunately, the $\rho^0$ mainly decays into $\pi^0+\pi^0$, not $\gamma+\pi^0$ and is therefore suppressed. This information is currently not known to the expert system, but it is possible to hand the expert system a list of allowed intermediate states.

In [ ]:
# particles are found by name comparison,
# i.e. f2 will find all f2's and f all f's independent of their spin
stm.set_allowed_intermediate_particles(["f(0)", "f(2)"])

result = stm.find_solutions(problem_sets)

print("found", len(result.transitions), "solutions!")
result.get_intermediate_particles().names

Now we have selected all amplitudes that involve **f** states. The warnings appear only to notify the user that the list of solutions is not exhaustive: for certain edges in the graph, no suitable particle was found (since only f states were allowed).

In [ ]:
import graphviz
from expertsystem import io

dot = io.asdot(result, collapse_graphs=True, render_node=False)
graphviz.Source(dot)

:::{seealso}

{doc}`/usage/visualize`

:::

## 4. Export generated transitions

The {class}`.Result`, {class}`.StateTransitionGraph`, and {class}`.Topology` can be serialized to and from a {obj}`dict` with {func}`.io.asdict` and {func}`.io.fromdict`:

In [ ]:
from expertsystem import io

graph = result.transitions[0]
io.asdict(graph.topology)

```{margin}
YAML is more human-readable than JSON, but reading and writing JSON is faster.
```

This also means that the {obj}`.Result` can be written to JSON or YAML format with {func}`.io.write` and loaded again with {func}`.io.load`:

In [ ]:
io.write(result, "transitions.json")
imported_result = io.load("transitions.json")
assert imported_result == result

Handy if it takes a lot of computation time to generate the transitions!

```{warning}
It's not possible to {mod}`pickle` a {class}`.Result`, because {class}`.StateTransitionGraph` makes use of {class}`~typing.Generic`.
```

## 5. Generate an amplitude model

````{margin} Formalism
In this example, we used the helicity formalism. If you want to work with the canonical formalism, you have to construct the {class}`.StateTransitionManager` with argument

```python
formalism_type="canonical-helicity"
```

instead of

```python
formalism_type="helicity"
```
````

Now that we are satisfied with the intermediate resonances, we are all set convert the solutions that the STM found to an amplitude model! This can be done with {func}`.get_builder`. This function deduces the amplitude formalism from the {attr}`~.StateTransitionManager.formalism_type` that you specified [when constructing the STM](#define-the-problem-set). Its output {class}`.HelicityAmplitudeBuilder` builder class can {meth}`.HelicityAmplitudeBuilder.generate` a {class}`.HelicityModel` that defines the mathematical formulation of an amplitude model in the helicity (or canonical) formalism.

In [ ]:
import expertsystem as es

model_builder = es.amplitude.get_builder(result)
amplitude_model = model_builder.generate()
next(iter(amplitude_model.components.values()))

From here on, have a look at {doc}`amplitude` to see how to work with the {class}`.HelicityModel` class.